In [1]:
import numpy as np
import os
import random
from sympy.solvers import solve
from sympy import Symbol
from sklearn.metrics import accuracy_score

In [2]:
model_1_score=np.load("C:\\Users\\User\\Desktop\\Sugeno Integral\\Confidence Score\\PAMPA2\\PAMAP_incep.npy")
model_2_score=np.load("C:\\Users\\User\\Desktop\\Sugeno Integral\\Confidence Score\\PAMPA2\\PAMAP_resnet.npy")
model_3_score=np.load("C:\\Users\\User\\Desktop\\Sugeno Integral\\Confidence Score\\PAMPA2\\PAMAP_cus.npy")
print(model_1_score.shape)
print(model_2_score.shape)
print(model_3_score.shape)

(4259, 12)
(4259, 12)
(4259, 12)


In [3]:
y_test=np.load('C:\\Users\\User\\Desktop\\Sugeno Integral\\Confidence Score\\PAMPA2\\PAMAP_y_test.npy')
#y_test=test_data['y_test']
print(y_test.shape)

(4259,)


In [4]:
def Function(x,y,z):
    fuzzy_measure=np.array([x,y,z])
    l=Symbol('l',real=True)
    lam=solve( ((1+l*fuzzy_measure[0])*(1+l*fuzzy_measure[1])*(1+l*fuzzy_measure[2]))-(1+l) , l )
    lam=[i for i in lam if i>0]
    if len(lam)==0:
        return 1
    lam.sort()
    lam=float(lam[0])
    #print("Lam value is : {}".format(lam))
    
    fuzzy_score=np.zeros(shape=model_1_score.shape, dtype=float)
    for i in range(fuzzy_score.shape[0]):
        for j in range(12):
            
            f_score=np.asarray([ model_1_score[i][j], model_2_score[i][j], model_3_score[i][j] ])
            
            permute_index=np.argsort(f_score)
            f_score_permuted=f_score[permute_index]
            fuzzy_measure_permuted=fuzzy_measure[permute_index]
            
            A_0=1.0
            A_1=( fuzzy_measure_permuted[1] + fuzzy_measure_permuted[2] + lam*fuzzy_measure_permuted[1]*fuzzy_measure_permuted[2] )
            A_2=fuzzy_measure_permuted[2]
            #print("The A_i values are {} , {}, {}".format(A_0,A_1,A_2))
            
            sugeno_score=max([ min(f_score_permuted[0],A_0), min(f_score_permuted[1],A_1), min(f_score_permuted[2],A_2) ])
            #print("Integral score is : {}".format(sugeno_score))
            
            fuzzy_score[i][j]=sugeno_score
            
    fuzzy_y_pred=np.argmax(fuzzy_score,axis=1)
    #print("shape is : {}".format(fuzzy_y_pred.shape))
    #print("argmax : {}".format(np.argmax(fuzzy_score[100])))
    acc=accuracy_score(y_test,fuzzy_y_pred)
    
    return (1-acc)

In [5]:
def Fitness(sample):
    fitness_score=Function(sample[0],sample[1],sample[2])
    if fitness_score == 0:
        return 9999
    else:
        return abs(1/fitness_score)

In [6]:
for i in range(100):
    
    population=[]
    for p in range(100):
        population.append( (random.uniform(0,1),
                            random.uniform(0,1),
                            random.uniform(0,1)) )  
    
    rank=[]
    for p in population:
        fitness=Fitness(p)
        rank.append( (fitness,p))
    rank.sort()
    rank.reverse()
    print(rank[0])
    
    best_population=rank[:25]
    
    if best_population[0][0] > 999:
        break
    
    all_candidate=[]
    for item in best_population:
        all_candidate.append( item[1][0])
        all_candidate.append(item[1][1])
        all_candidate.append(item[1][2])
    
    new_gen=[]
    for _ in range(100):
        e1=random.choice(all_candidate) * random.uniform(0.1,0.3)
        e2=random.choice(all_candidate) * random.uniform(0.1,0.3)
        e3=random.choice(all_candidate) * random.uniform(0.1,0.3)
        new_gen.append( (e1,e2,e3) )
    
    population=new_gen    

(56.78666666666657, (0.2629315340924542, 0.10425421490344111, 0.407491083200198))
(59.985915492957886, (0.24003273079049237, 0.16381235106959535, 0.28828389002957655))
(61.724637681159486, (0.15051274000106019, 0.14832395051062053, 0.5177078581655473))
(58.342465753424484, (0.2925894888488023, 0.3613720753766808, 0.201428853708571))
(57.554054054054106, (0.15580151898421446, 0.10924347783830912, 0.6216662936424888))
(58.342465753424484, (0.22318445590464242, 0.22022095076649162, 0.37569808441667096))
(59.15277777777776, (0.6475430653591908, 0.14730302825942132, 0.19447362875909757))
(58.342465753424484, (0.33318661813385053, 0.3208118795225623, 0.20005175996225022))
(56.78666666666657, (0.3025996655220491, 0.00030627946323535937, 0.626705713978607))
(59.15277777777776, (0.19379388796998465, 0.3287692163269962, 0.30407844977842347))
(59.15277777777776, (0.34342149122952237, 0.15094327669353402, 0.18484578422045272))
(58.342465753424484, (0.33370449167381866, 0.2516660063051853, 0.243203

(60.84285714285704, (0.3422475903819384, 0.14462511808389922, 0.2534708319069191))
(60.84285714285704, (0.4835895192004006, 0.1376086757408258, 0.2701914424052455))
